In [1]:
import re
import os
from datetime import datetime

def parse_filename(full_path):
    filename = os.path.basename(full_path)

    # Extracting label from the directory path (assuming it's part of the path)
    path_parts = full_path.split('\\')
    label = path_parts[-2]  # Assuming the label is the second last part of the path

    # Updated regular expression to handle both 'time_point-xx' and a numeric pattern
    pattern = r'(\d{4})_(\d{2})_(\d{2})_(\d{2})h(\d{2})_Plate#(\d+)(?:_([a-zA-Z0-9-]+)|_time_point-(\d+))_AcquisitionBlock(\d+)_pt(\d+)_10x_stitched_Scene(\d+).png'
    match = re.search(pattern, filename)

    if match:
        year, month, day, hour, minute, plate, id_or_time_point, time_point, block, pt, scene = match.groups()
        timestamp = datetime(int(year), int(month), int(day), int(hour), int(minute))

        # Determine if the identifier is a time_point or an ID
        identifier = time_point if time_point else id_or_time_point

        return {
            'timestamp': timestamp,
            'plate': plate,
            'identifier': identifier,  # Can be either an ID or a time_point
            'block': block,
            'pt': pt,
            'scene': scene,
            'label': label
        }
    else:
        return None

In [5]:
import os
import shutil
import csv

os.path.split
# Set the directory you want to start from
rootDir = r'E:\New_data_Maxim\images_classified'
outputDir = r'E:\New_data_Maxim\gifs'
file_paths = []
for dirName, subdirList, fileList in os.walk(rootDir):
    label = str(os.path.split(dirName)[-1])
    if label == "images_classified":
        pass
    for fname in fileList:
        if ".png" in fname:
            path = os.path.join(rootDir, label, fname)
            file_paths.append(path)


# Creating a dictionary with paths grouped by similar names except the date, and sorted by date
grouped_sorted_paths = {}
for path in file_paths:
    parsed = parse_filename(path)
    if parsed:
        # Create a key excluding the date
        key = (parsed['plate'], parsed['block'], parsed['pt'], parsed['scene'])
        if key not in grouped_sorted_paths:
            grouped_sorted_paths[key] = []
        grouped_sorted_paths[key].append((parsed['timestamp'], parsed['label'], path))

# Sorting the paths in each group by date
for key in grouped_sorted_paths:
    grouped_sorted_paths[key].sort()

# Using the function in the loop
csv_file = "E:/New_data_Maxim/croped_images/time_line_for_deep_learning.csv"
# Open the file for writing
with open(csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header row, if needed
    csvwriter.writerow(['Previous Label', 'Previous Image Path', 'Current Label', 'Current Image Path'])

    for key, values in grouped_sorted_paths.items():
        # Extract labels and file paths
        image_info = [(value[1], value[2]) for value in values]

        # Iterate over pairs of consecutive images
        for i in range(len(image_info) - 1):
            previous_image = image_info[i]
            current_image = image_info[i + 1]

            # Write data to CSV
            csvwriter.writerow([previous_image[0], previous_image[1], current_image[0], current_image[1]])



            